In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv(r"..\Data Cleaning\smartphones_feature_selection_cleaned.csv")

In [7]:
df.head()

,brand_name,rating,has_5g,has_nfc,processor_brand,processor_speed,ram_capacity,internal_memory,battery_capacity,fast_charging,num_rear_cameras,screen_size,resolution,refresh_rate,os,extended_upto,primary_camera_rear,primary_camera_front,price
0,22.0,89,1,1,1.0,3.2,12,256,4500,67,2,6.78,0.0,120,2.0,0,50,32,79990
1,28.0,82,0,1,5.0,2.2,8,256,5000,33,3,6.67,0.0,120,0.0,0,64,32,16990
2,31.0,82,1,0,1.0,2.6,8,256,5000,67,3,6.67,0.0,120,0.0,0,50,16,23799
3,25.0,75,1,0,1.0,2.2,4,128,5000,18,3,6.50,0.0,90,0.0,1024,48,16,16999
4,31.0,74,1,0,11.0,2.0,4,64,4800,18,3,6.50,0.0,90,0.0,1024,48,8,10490


In [14]:
df.columns

Index(['brand_name', 'rating', 'has_5g', 'has_nfc', 'processor_brand',
       'processor_speed', 'ram_capacity', 'internal_memory',
       'battery_capacity', 'fast_charging', 'num_rear_cameras', 'screen_size',
       'resolution', 'refresh_rate', 'os', 'extended_upto',
       'primary_camera_rear', 'primary_camera_front', 'price'],
      dtype='object')

In [9]:
# One hot encode -> brand_name, processor_brand, resolution, os

In [10]:
X = df.drop(columns=['price'])
y = df['price']

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [12]:
columns_to_encode = ['brand_name','processor_brand','resolution','os','has_5g','has_nfc']

In [13]:
y_transformed = np.log1p(y)

In [40]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['rating', 'processor_speed', 'ram_capacity', 'internal_memory','battery_capacity',
        'fast_charging', 'num_rear_cameras', 'screen_size','refresh_rate', 'extended_upto','primary_camera_rear', 'primary_camera_front']),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [41]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [42]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

C:\Users\Joel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Joel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Joel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [43]:
scores.mean()

0.9149370729326712

In [44]:
scores.std()

0.013257539337660322

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [46]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['rating', 'processor_speed',
                                                   'ram_capacity',
                                                   'internal_memory',
                                                   'battery_capacity',
                                                   'fast_charging',
                                                   'num_rear_cameras',
                                                   'screen_size',
                                                   'refresh_rate',
                                                   'extended_upto',
                                                   'primary_camera_rear',
                                                   'primary_camera_front']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['brand_name',
                                                   'processor_brand',
                                                   'resolution', 'os'])])),
                ('regressor', SVR())])

In [47]:
y_pred = pipeline.predict(X_test)

C:\Users\Joel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [48]:
y_pred = np.expm1(y_pred)

In [49]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

5867.093289486394